# Importing libraries

In [45]:
from pyzotero import zotero
import os
# import tweepy as tw
import pandas as pd
import datetime
import json, sys
from datetime import date, timedelta  
import datetime
import plotly.express as px
import pycountry
import re
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

In [46]:
"# All items in the Zotero Intelligence bibliography library".index("All")

2

In [47]:
library_id = '2514686'
library_type = 'group'
api_key = '' # api_key is only needed for private groups and libraries
zot = zotero.Zotero(library_id, library_type)


# All items in the Zotero Intelligence bibliography library

In [48]:
items = zot.everything(zot.top())

data3=[]
columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key', 'DOI', 'Book_title', 'Thesis_type', 'University']

for item in items:
    # Extracting various types of creators
    authors = ""
    if 'creators' in item['data']:
        authors_info = item['data']['creators']
        author_names = []
        for author in authors_info:
            creator_type = author.get('creatorType')
            valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster', 'presenter', 'translator', 'programmer']
            
            if item['data']['itemType'] == 'bookSection' and creator_type == 'editor':
                continue  # Skip adding 'editor' for Book chapters
            
            if creator_type in valid_types:
                if 'firstName' in author and 'lastName' in author:
                    author_names.append(author['firstName'] + ' ' + author['lastName'])
                elif 'name' in author:
                    author_names.append(author['name'])
                
        authors = ', '.join(author_names)
        
    data3.append((
        item['data']['title'], 
        item['data']['itemType'], 
        item['data']['url'], 
        item['data']['abstractNote'], 
        item['links']['alternate']['href'],
        item['data'].get('date'),
        authors,  # Add the concatenated authors string
        item['data'].get('publisher'),
        item['data'].get('publicationTitle'),
        item['data']['dateAdded'],
        item['data']['collections'],
        item['data'].get('DOI'),
        item['data'].get('bookTitle'),
        item['data'].get('thesisType', ''),
        item['data'].get('university', '')
        ))
# pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data3, columns=columns3)

mapping_types = {
    'thesis': 'Thesis',
    'journalArticle': 'Journal article',
    'book': 'Book',
    'bookSection': 'Book chapter',
    'blogPost': 'Blog post',
    'videoRecording': 'Video',
    'podcast': 'Podcast',
    'magazineArticle': 'Magazine article',
    'webpage': 'Webpage',
    'newspaperArticle': 'Newspaper article',
    'report': 'Report',
    'forumPost': 'Forum post',
    'manuscript': 'Manuscript',
    'document': 'Document',
    'conferencePaper': 'Conference paper',
    'film': 'Film',
    'presentation': 'Presentation',
    'audioRecording':'Podcast',
    'preprint':'Preprint',
    'hearing':'Hearing',
    'computerProgram':'Computer program',
    'dataset':'Dataset'
}
df['Publication type'] = df['Publication type'].replace(mapping_types)

mapping_publisher = {
    'Taylor & Francis Group': 'Taylor and Francis',
    'Taylor and Francis': 'Taylor and Francis',
    'Taylor & Francis': 'Taylor and Francis',
    'Routledge': 'Routledge',
    'Routledge Handbooks Online': 'Routledge',
    'Praeger Security International': 'Praeger',
    'Praeger': 'Praeger'
}
df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

mapping_journal = {
    'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and Counter Intelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'Intelligence and national security': 'Intelligence and National Security',
    'Intelligence and National Security': 'Intelligence and National Security',
    'Intelligence & National Security': 'Intelligence and National Security'
}

df['Journal'] = df['Journal'].replace(mapping_journal)

mapping_thesis_type ={
    "MA Thesis": "Master's Thesis",
    "PhD Thesis": "PhD Thesis",
    "Master Thesis": "Master's Thesis",
    "Thesis": "Master's Thesis",  # Assuming 'Thesis' refers to Master's Thesis here, adjust if necessary
    "Ph.D.": "PhD Thesis",
    "Master's Dissertation": "Master's Thesis",
    "Undergraduate Theses": "Undergraduate Thesis",
    "MPhil": "MPhil Thesis",
    "A.L.M.": "Master's Thesis",  # Assuming A.L.M. (Master of Liberal Arts) maps to Master's Thesis
    "doctoralThesis": "PhD Thesis",
    "PhD": "PhD Thesis",
    "Masters": "Master's Thesis",
    "PhD thesis": "PhD Thesis",
    "phd": "PhD Thesis",
    "doctoral": "PhD Thesis",
    "Doctoral": "PhD Thesis",
    "Master of Arts Dissertation": "Master's Thesis",
    "":'Unclassified'
}
df['Thesis_type'] = df['Thesis_type'].replace(mapping_thesis_type)
# df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [49]:
df_test = df[df['Title']=='Sir Robert Cecil and Elizabethan Intelligencing, 1590-1603']
df_test

,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Book_title,Thesis_type,University
6050,Sir Robert Cecil and Elizabethan Intelligencin...,Thesis,http://oro.open.ac.uk/78595/,Much scholarly attention has been given to the...,https://www.zotero.org/groups/intelligence_bib...,2021-08-04,Christopher Mains,None,None,2022-12-28T12:32:53Z,[DS3WDJUS],None,None,PhD Thesis,The Open University


In [50]:
# items = zot.everything(zot.top())

# data3=[]
# columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key']

# for item in items:
#     # Extracting various types of creators
#     authors = ""
#     if 'creators' in item['data']:
#         authors_info = item['data']['creators']
#         author_names = []
#         for author in authors_info:
#             creator_type = author.get('creatorType')
#             valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster']
#             if creator_type in valid_types:  # Include all identified creator types
#                 if 'firstName' in author and 'lastName' in author:
#                     author_names.append(author['firstName'] + ' ' + author['lastName'])
#                 elif 'name' in author:
#                     author_names.append(author['name'])  # Handling cases where 'firstName' and 'lastName' are absent
                
#         authors = ', '.join(author_names)
        
#     data3.append((
#         item['data']['title'], 
#         item['data']['itemType'], 
#         item['data']['url'], 
#         item['data']['abstractNote'], 
#         item['links']['alternate']['href'],
#         item['data'].get('date'),
#         authors,  # Add the concatenated authors string
#         item['data'].get('publisher'),
#         item['data'].get('publicationTitle'),
#         item['data']['dateAdded'],
#         item['data']['collections']
#         )) 
# # pd.set_option('display.max_colwidth', None)

# df = pd.DataFrame(data3, columns=columns3)

# mapping_types = {
#     'thesis': 'Thesis',
#     'journalArticle': 'Journal article',
#     'book': 'Book',
#     'bookSection': 'Book chapter',
#     'blogPost': 'Blog post',
#     'videoRecording': 'Video',
#     'podcast': 'Podcast',
#     'magazineArticle': 'Magazine article',
#     'webpage': 'Webpage',
#     'newspaperArticle': 'Newspaper article',
#     'report': 'Report',
#     'forumPost': 'Forum post',
#     'manuscript': 'Manuscript',
#     'document': 'Document',
#     'conferencePaper': 'Conference paper',
#     'film': 'Film',
#     'presentation': 'Presentation',
#     'audioRecording':'Podcast'
# }
# df['Publication type'] = df['Publication type'].replace(mapping_types)

# mapping_publisher = {
#     'Taylor & Francis Group': 'Taylor and Francis',
#     'Taylor and Francis': 'Taylor and Francis',
#     'Taylor & Francis': 'Taylor and Francis',
#     'Routledge': 'Routledge',
#     'Routledge Handbooks Online': 'Routledge',
#     'Praeger Security International': 'Praeger',
#     'Praeger': 'Praeger'
# }
# df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# # df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# # df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# # df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

# mapping_journal = {
#     'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'Intelligence and national security': 'Intelligence and National Security',
#     'Intelligence and National Security': 'Intelligence and National Security',
#     'Intelligence & National Security': 'Intelligence and National Security'
# }

# df['Journal'] = df['Journal'].replace(mapping_journal)

# # df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# # df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [51]:
def zotero_collections2(library_id, library_type):
    collections = zot.collections()
    data = [(item['data']['key'], item['data']['name'], item['meta']['numItems'], item['links']['alternate']['href']) for item in collections]
    df_collections = pd.DataFrame(data, columns=['Key', 'Name', 'Number', 'Link'])
    return df_collections
df_collections_2 = zotero_collections2(library_id, library_type)

In [52]:
def duplicate_rows_by_col_key(df, df_collections):
    # Duplicate rows based on 'Col key'
    duplicated_rows = []
    for index, row in df.iterrows():
        if isinstance(row['Col key'], list):
            for key in row['Col key']:
                new_row = row.copy()
                collection_info = df_collections[df_collections['Key'] == key]
                if not collection_info.empty:
                    new_row['Collection_Key'] = key
                    new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                    new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                    duplicated_rows.append(new_row)
        else:
            key = row['Col key']
            new_row = row.copy()
            collection_info = df_collections[df_collections['Key'] == key]
            if not collection_info.empty:
                new_row['Collection_Key'] = key
                new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                duplicated_rows.append(new_row)

    # Create a new DataFrame with duplicated rows
    duplicated_df = pd.DataFrame(duplicated_rows)

    return duplicated_df

# Duplicating rows based on 'Col key' and collections information
duplicated_df = duplicate_rows_by_col_key(df, df_collections_2)

In [53]:
# day_allitems = datetime.date.today().isoformat()

In [54]:
# df_fa = df['FirstName2']
# df_fa = pd.DataFrame(df_fa.tolist())
# df_fa = df_fa[0]
# df_fa = df_fa.apply(lambda x: {} if pd.isna(x) else x)
# df_new = pd.json_normalize(df_fa, errors='ignore') 
# df = pd.concat([df, df_new], axis=1)
# df['firstName'] = df['firstName'].fillna('null')
# df['lastName'] = df['lastName'].fillna('null')

In [55]:
df.to_csv('all_items.csv')
duplicated_df.to_csv('all_items_duplicated.csv')

In [56]:
# import psycopg2
# from sqlalchemy import create_engine
# import psycopg2.extras as extras

# Countries 

In [57]:
# Read the csv file into a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')

# Dictionary to map non-proper country names to their proper names
country_map = {
    'british': 'UK',
    'great britain': 'UK',
    'UK' : 'UK', 
    'america' : 'United States',
    'United States of America' : 'United States',
    'Soviet Union': 'Russia', 
    'american' : 'United States',
    'United States' : 'United States',
    'russian' : 'Russia'
    # Add more mappings as needed
}

# Find the country names in the "title" column of the dataframe
found_countries = {}
for i, row in df.iterrows():
    title = str(row['Title']).lower()
    for country in pycountry.countries:
        name = country.name.lower()
        if name in title or (name + 's') in title:  # Check for singular and plural forms of country names
            proper_name = country.name
            found_countries[proper_name] = found_countries.get(proper_name, 0) + 1
    for non_proper, proper in country_map.items():
        if non_proper in title:
            found_countries[proper] = found_countries.get(proper, 0) + title.count(non_proper)

# Create a new dataframe containing the found countries and their counts
df_countries = pd.DataFrame({'Country': list(found_countries.keys()), 'Count': list(found_countries.values())})

In [58]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [59]:
df_countries=df_countries.sort_values(by='Count', ascending=False)
df_countries.reset_index(drop=True)

,Country,Count
0,United States,710
1,UK,480
2,Russia,161
3,Israel,154
4,Ukraine,129
...,...,...
132,Lesotho,1
133,Madagascar,1
134,Mauritania,1
135,Mauritius,1


In [60]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [61]:
df_countries.to_csv('countries.csv',index=False)

# NER analysis

In [62]:
import spacy
import nltk
import ast
from spacy.pipeline import EntityRecognizer

In [63]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    # Download the model if not present
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MI6"}]
ruler.add_patterns(patterns)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
c:\Users\yaozk\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:865: UserWarning:

[W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible

In [64]:
def extract_entities(text):
    doc = nlp(text)
    orgs = []
    gpes = []
    people = []
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            orgs.append(entity.text)
        elif entity.label_ == 'GPE':
            gpes.append(entity.text)
        elif entity.label_ == 'PERSON':
            people.append(entity.text)
    return pd.Series({'ORG': orgs, 'GPE': gpes, 'PERSON': people})

In [65]:
df_title = df[['Title']].copy()
df_title = df_title.rename(columns={'Title':'Text'})
df_abstract = df[['Abstract']].copy()
df_abstract = df_abstract.rename(columns={'Abstract':'Text'})
df_one = pd.concat([df_title, df_abstract], ignore_index=True)
df_one['Text'] = df_one['Text'].fillna('')

In [66]:
df_one = pd.concat([df_one[['Text']], df_one['Text'].apply(extract_entities)], axis=1)
df_one = df_one.explode('GPE').reset_index(drop=True)
df_one = df_one.explode('ORG').reset_index(drop=True)
df_one = df_one.explode('PERSON').reset_index(drop=True)

In [67]:
df_one_g = df_one.copy()
df_one_g = df_one[['Text', 'GPE']]
# df_one_g = df_one_g.fillna('')
df_one_g = df_one_g.drop_duplicates(subset=['Text', 'GPE'])

gpe_counts = df_one_g['GPE'].value_counts().reset_index()
gpe_counts.columns = ['GPE', 'count']

# pd.options.display.max_rows = None


mapping_locations = {
    'the United States': 'USA',
    'The United States': 'USA',
    'US': 'USA',
    'U.S.': 'USA',
    'United States' : 'USA',
    'America' : 'USA',
    'the United States of America' : 'USA',
    'Britain' : 'UK',
    'the United Kingdom': 'UK',
    'U.K.' : 'UK',
    'Global Britain' : 'UK',
    'United Kingdom' : 'UK', 
    'the Soviet Union' : 'Russia',
    'The Soviet Union' : 'Russia',
    'USSR' : 'Russia',
    'Ukraine - Perspective' : 'Ukraine',
    'Ukrainian' : 'Ukraine',
    'Great Britain' : 'UK',
    'Ottoman Empire' : 'Turkey'
}
gpe_counts['GPE'] =gpe_counts['GPE'].replace(mapping_locations)
gpe_counts = gpe_counts.groupby('GPE').sum().reset_index()
gpe_counts.sort_values('count', ascending=False, inplace=True)
gpe_counts = gpe_counts.reset_index(drop=True)
gpe_counts.head(15)

,GPE,count
0,USA,1543
1,UK,785
2,Russia,580
3,Germany,233
4,Ukraine,232
5,Israel,189
6,China,179
7,France,158
8,Canada,150
9,Iraq,142


In [68]:
df_one_p = df_one.copy()
df_one_p = df_one[['Text', 'PERSON']]
# df_one_p = df_one_g.fillna('')
df_one_p = df_one_p.drop_duplicates(subset=['Text', 'PERSON'])

person_counts = df_one_p['PERSON'].value_counts().reset_index()
person_counts.columns = ['PERSON', 'count']

mapping_person = {
    'Putin' : 'Vladimir Putin',
    'Vladimir Putin' : 'Vladimir Putin',
    'Churchill' : 'Winston Churchill',
    'Hitler' : 'Adolf Hitler',
    'Biden' : 'Joe Biden',
    "John le Carré’s" : "John le Carré"
}

person_counts['PERSON'] =person_counts['PERSON'].replace(mapping_person)
person_counts = person_counts.groupby('PERSON').sum().reset_index()
person_counts.sort_values('count', ascending=False, inplace=True)

remove_person = ['MI6', 'Twitter', 'GRU'
          ]
person_counts = person_counts[~person_counts['PERSON'].isin(remove_person)]
person_counts = person_counts.reset_index(drop=True)

person_counts.head(15)

,PERSON,count
0,Adolf Hitler,90
1,Vladimir Putin,80
2,Winston Churchill,57
3,Andrew Hammond,56
4,Stalin,43
5,James Bond,30
6,Eisenhower,29
7,Mujibur Rahman,28
8,Bush,28
9,John le Carré,27


In [69]:
df_one_o = df_one.copy()
df_one_o = df_one[['Text', 'ORG']]
# df_one_p = df_one_g.fillna('')
df_one_o = df_one_o.drop_duplicates(subset=['Text', 'ORG'])

org_counts = df_one_o['ORG'].value_counts().reset_index()
org_counts.columns = ['ORG', 'count']

mapping_organisations = {
    'The British Secret Intelligence Service' : 'SIS',
    'the British Secret Intelligence Service' : 'SIS',
    'The Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee - History' : 'Joint Intelligence Committee',
    'Central Intelligence Agency' : 'CIA',
    'the Central Intelligence Agency' : 'CIA',
    'the Foreign Office' : 'Foreign Office',
    'Schar School' : 'Schar School of Policy and Government',
    'the Secret Intelligence Service' : 'SIS',
    "George Mason University's" : "George Mason University",
    'JIC' : 'Joint Intelligence Committee'
}
org_counts['ORG'] =org_counts['ORG'].replace(mapping_organisations)
org_counts = org_counts.groupby('ORG').sum().reset_index()
org_counts.sort_values('count', ascending=False, inplace=True)

remove_orgs = ['Intelligence', 'Kremlin', 'Ultra', 'International Security', 'Intelligence Studies', 'Intelligence Analysis', 'OSINT']
org_counts = org_counts[~org_counts['ORG'].isin(remove_orgs)]
org_counts = org_counts.reset_index(drop=True)

org_counts.head(15)

,ORG,count
0,CIA,780
1,KGB,155
2,FBI,115
3,Allied,107
4,Congress,86
5,SIS,73
6,NATO,70
7,the Intelligence Community,69
8,SOE,67
9,EU,59


In [70]:
gpe_counts.head(15).to_csv('gpe.csv')
person_counts.head(15).to_csv('person.csv')
org_counts.head(15).to_csv('org.csv')

In [71]:
test = pd.read_csv('all_items.csv')
test

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Book_title,Thesis_type,University
0,0,Intelligence Officers Have an Ethical Responsi...,Journal article,https://doi.org/10.1080/08850607.2024.2381999,Intelligence officers have an ethical responsi...,https://www.zotero.org/groups/intelligence_bib...,2024-08-29,Cortney Weinbaum,NaN,Intl Journal of Intelligence and Counterintell...,2024-09-03T07:10:44Z,['DVEM4H4W'],10.1080/08850607.2024.2381999,NaN,Unclassified,NaN
1,1,12. SOS Hong Kong: Coproducing Espionage Films...,Book chapter,https://www.degruyter.com/document/doi/10.1515...,With the success of D r. No and Goldfinger in ...,https://www.zotero.org/groups/intelligence_bib...,2024-07-10,Sangjoon Lee,Amsterdam University Press,NaN,2024-09-03T07:09:02Z,['9YH9YSYQ'],NaN,Remapping the Cold War in Asian Cinemas,Unclassified,NaN
2,2,Counterintelligence Analysis: Catching Spies a...,Book chapter,http://press.georgetown.edu/Book/The-CIA-Intel...,NaN,https://www.zotero.org/groups/intelligence_bib...,2024-09-01,"James B. Bruce, Blake W. Mobley",Georgetown University Press,NaN,2024-09-02T08:17:28Z,"['CZJ36V8L', 'RHJFPRAI']",NaN,The CIA Intelligence Analyst: Views from the I...,Unclassified,NaN
3,3,The CIA Intelligence Analyst: Views from the I...,Book,http://press.georgetown.edu/Book/The-CIA-Intel...,A unique insiders' account of what CIA intelli...,https://www.zotero.org/groups/intelligence_bib...,2024-09-01,"Roger Z. George, Robert Levine",Georgetown University Press,NaN,2024-09-02T08:14:08Z,['CZJ36V8L'],NaN,NaN,Unclassified,NaN
4,4,Intelligence and State Surveillance in Modern ...,Book,https://www.emerald.com/insight/publication/do...,NaN,https://www.zotero.org/groups/intelligence_bib...,2024-09-13,Frederic Lemieux,Emerald Publishing Limited,NaN,2024-08-31T18:43:31Z,['28B8SB3Y'],NaN,NaN,Unclassified,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7192,7192,The Uses and Limits of Intelligence,Book,https://doi.org/10.4324/9781351302128,NaN,https://www.zotero.org/groups/intelligence_bib...,2017,Walter Laqueur,Routledge,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN,Unclassified,NaN
7193,7193,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN,Unclassified,NaN
7194,7194,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN,Unclassified,NaN
7195,7195,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN,Unclassified,NaN


# Retrieving citation count and open access status

In [72]:
df_doi = pd.read_csv('all_items.csv')

df_doi = df_doi[['Zotero link', 'DOI']].dropna()
df_doi = df_doi.drop(df_doi[df_doi['DOI'] == ''].index)
df_doi = df_doi.reset_index(drop=True)
df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')

def fetch_article_metadata(doi):
    base_url = 'https://api.openalex.org/works/https://doi.org/'
    response = requests.get(base_url + doi)
    if response.status_code == 200:
        data = response.json()
        counts_by_year = data.get('counts_by_year', [])
        if counts_by_year:
            first_citation_year = min(entry.get('year') for entry in data['counts_by_year'])
        else:
            first_citation_year = None
        if data.get('counts_by_year'):
            last_citation_year = max(entry.get('year') for entry in data['counts_by_year'])
        else:
            last_citation_year = None

        article_metadata = {
            'ID': data.get('id'),
            'Citation': data.get('cited_by_count'),
            'OA status': data.get('open_access', {}).get('is_oa'),
            'Citation_list': data.get('cited_by_api_url'),
            'First_citation_year': first_citation_year,
            'Last_citation_year': last_citation_year,
            'Publication_year': data.get('publication_year'),
            'OA_link': data.get('open_access', {}).get('oa_url')
        }
        return article_metadata
    else:
        return {
            'ID': None,
            'Citation': None,
            'OA status': None,
            'First_citation_year': None,
            'Last_citation_year': None,
            'Publication_year': None,
            'OA_link': None
        }

df_doi['ID'] = None
df_doi['Citation'] = None
df_doi['OA status'] = None
df_doi['Citation_list'] = None
df_doi['First_citation_year'] = None
df_doi['Last_citation_year'] = None
df_doi['Publication_year'] = None
df_doi['OA_link'] = None

# Iterate over each row in the DataFrame
for index, row in df_doi.iterrows():
    doi = row['DOI']
    article_metadata = fetch_article_metadata(doi)
    if article_metadata:
        # Update DataFrame with fetched information
        df_doi.at[index, 'ID'] = article_metadata['ID']
        df_doi.at[index, 'Citation'] = article_metadata['Citation']
        df_doi.at[index, 'OA status'] = article_metadata['OA status']
        df_doi.at[index, 'First_citation_year'] = article_metadata['First_citation_year']
        df_doi.at[index, 'Last_citation_year'] = article_metadata['Last_citation_year']
        df_doi.at[index, 'Citation_list'] = article_metadata.get('Citation_list', None)
        df_doi.at[index, 'Publication_year'] = article_metadata['Publication_year']
        df_doi.at[index, 'OA_link'] = article_metadata.get('OA_link', None)

# Calculate the difference between First_citation_year and Publication_year
df_doi['Year_difference'] = df_doi['First_citation_year'] - df_doi['Publication_year']
df_doi.to_csv('citations.csv')

df_doi

C:\Users\yaozk\AppData\Local\Temp\ipykernel_31328\1235606195.py:7: FutureWarning:

The default value of regex will change from True to False in a future version.



,Zotero link,DOI,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2381999,https://openalex.org/W4401989597,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
1,https://www.zotero.org/groups/intelligence_bib...,10.1093/fh/crae020,https://openalex.org/W4401441472,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
2,https://www.zotero.org/groups/intelligence_bib...,10.1080/16161262.2024.2387982,https://openalex.org/W4401893695,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
3,https://www.zotero.org/groups/intelligence_bib...,10.3201/eid3009.240082,https://openalex.org/W4401814697,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
4,https://www.zotero.org/groups/intelligence_bib...,10.1177/17506352241274554,https://openalex.org/W4401846574,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3669,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,7,True,https://api.openalex.org/works?filter=cites:W2...,2018,2024,2017,https://journals.sagepub.com/doi/pdf/10.1177/1...,1
3670,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,6,False,https://api.openalex.org/works?filter=cites:W2...,2021,2024,2019,None,2
3671,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,30,False,https://api.openalex.org/works?filter=cites:W2...,2013,2023,2013,None,0
3672,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,1,False,https://api.openalex.org/works?filter=cites:W3...,2024,2024,2020,None,4


In [73]:
# ## Remove after the first successful retrieval 2024-02-18

# df_doi = pd.read_csv('all_items.csv')
# if datetime.today().weekday() == 3:

#     df_doi = df_doi[['Zotero link', 'DOI']].dropna()
#     df_doi = df_doi.drop(df_doi[df_doi['DOI'] == ''].index)
#     df_doi = df_doi.reset_index(drop=True)
#     df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')

#     def fetch_article_metadata(doi):
#         base_url = 'https://api.openalex.org/works/https://doi.org/'
#         response = requests.get(base_url + doi)
#         if response.status_code == 200:
#             data = response.json()
#             counts_by_year = data.get('counts_by_year', [])
#             if counts_by_year:
#                 first_citation_year = min(entry.get('year') for entry in data['counts_by_year'])
#             else:
#                 first_citation_year = None
#             article_metadata = {
#                 'ID': data.get('id'),
#                 'Citation': data.get('cited_by_count'),
#                 'OA status': data.get('open_access', {}).get('is_oa'),
#                 'Citation_list': data.get('cited_by_api_url'),
#                 'First_citation_year': first_citation_year,
#                 'Publication_year':data.get('publication_year')
#             }
#             return article_metadata
#         else:
#             return {'ID': None, 'Citation': None, 'OA status': None, 'First_citation_year': None, 'Publication_year' : None}
#     df_doi['ID'] = None
#     df_doi['Citation'] = None
#     df_doi['OA status'] = None
#     df_doi['Citation_list'] = None
#     df_doi['First_citation_year'] = None
#     df_doi['Publication_year'] = None

#     # Iterate over each row in the DataFrame
#     for index, row in df_doi.iterrows():
#         doi = row['DOI']
#         article_metadata = fetch_article_metadata(doi)
#         if article_metadata:
#             # Update DataFrame with fetched information
#             df_doi.at[index, 'ID'] = article_metadata['ID']
#             df_doi.at[index, 'Citation'] = article_metadata['Citation']
#             df_doi.at[index, 'OA status'] = article_metadata['OA status']
#             df_doi.at[index, 'First_citation_year'] = article_metadata['First_citation_year']
#             if 'Citation_list' in article_metadata:
#                 df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
#             else:
#                 # Set to None if key is not found
#                 df_doi.at[index, 'Citation_list'] = None
#             df_doi.at[index, 'Publication_year'] = article_metadata['Publication_year']

#     # Calculate the difference between First_citation_year and Publication_year
#     df_doi['Year_difference'] = df_doi['First_citation_year'] - df_doi['Publication_year']
#     df_doi.to_csv('citations.csv')
# else:
#     print('This part will be refreshed every Wednesday!')
# df_doi

In [74]:
# ## Retrieving citation and open access status without first ciation year
# df_doi = pd.read_csv('all_items.csv')
# # df_doi = df_doi.head(10)
# if datetime.today().weekday() == 5:

#     df_doi = df_doi[['Zotero link', 'DOI']].dropna()
#     df_doi = df_doi.drop(df_doi[df_doi['DOI']==''].index)
#     df_doi = df_doi.reset_index(drop=True)
#     df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')
#     df_doi

#     def fetch_article_metadata(doi):
#         base_url = 'https://api.openalex.org/works/https://doi.org/'
#         response = requests.get(base_url + doi)
#         if response.status_code == 200:
#             data = response.json()
#             article_metadata = {
#                 'ID': data.get('id'),
#                 'Citation': data.get('cited_by_count'),
#                 'OA status': data.get('open_access', {}).get('is_oa'),
#                 'Citation_list':data.get('cited_by_api_url')
#             }
#             return article_metadata
#         else:
#             return {'ID': None, 'Citation': None, 'OA status': None}

#     df_doi['ID'] = None
#     df_doi['Citation'] = None
#     df_doi['OA status'] = None
#     df_doi['Citation_list'] = None

#     # Iterate over each row in the DataFrame
#     for index, row in df_doi.iterrows():
#         doi = row['DOI']
#         article_metadata = fetch_article_metadata(doi)
#         if article_metadata:
#             # Update DataFrame with fetched information
#             df_doi.at[index, 'ID'] = article_metadata['ID']
#             df_doi.at[index, 'Citation'] = article_metadata['Citation']
#             df_doi.at[index, 'OA status'] = article_metadata['OA status']
#             if 'Citation_list' in article_metadata:
#                 df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
#             else:
#                 # Set to None if key is not found
#                 df_doi.at[index, 'Citation_list'] = None
#     df_doi
#     df_doi.to_csv('citations.csv')
# else:
#     print('This part will be refreshed every Wednesday!')

In [75]:
df_doi

,Zotero link,DOI,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2381999,https://openalex.org/W4401989597,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
1,https://www.zotero.org/groups/intelligence_bib...,10.1093/fh/crae020,https://openalex.org/W4401441472,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
2,https://www.zotero.org/groups/intelligence_bib...,10.1080/16161262.2024.2387982,https://openalex.org/W4401893695,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
3,https://www.zotero.org/groups/intelligence_bib...,10.3201/eid3009.240082,https://openalex.org/W4401814697,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
4,https://www.zotero.org/groups/intelligence_bib...,10.1177/17506352241274554,https://openalex.org/W4401846574,0,False,https://api.openalex.org/works?filter=cites:W4...,None,None,2024,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3669,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,7,True,https://api.openalex.org/works?filter=cites:W2...,2018,2024,2017,https://journals.sagepub.com/doi/pdf/10.1177/1...,1
3670,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,6,False,https://api.openalex.org/works?filter=cites:W2...,2021,2024,2019,None,2
3671,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,30,False,https://api.openalex.org/works?filter=cites:W2...,2013,2023,2013,None,0
3672,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,1,False,https://api.openalex.org/works?filter=cites:W3...,2024,2024,2020,None,4


In [76]:
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2

,Zotero link,DOI,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2381999,https://openalex.org/W4401989597,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
1,https://www.zotero.org/groups/intelligence_bib...,10.1093/fh/crae020,https://openalex.org/W4401441472,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
2,https://www.zotero.org/groups/intelligence_bib...,10.1080/16161262.2024.2387982,https://openalex.org/W4401893695,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
3,https://www.zotero.org/groups/intelligence_bib...,10.3201/eid3009.240082,https://openalex.org/W4401814697,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
4,https://www.zotero.org/groups/intelligence_bib...,10.1177/17506352241274554,https://openalex.org/W4401846574,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3669,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,7.0,True,https://api.openalex.org/works?filter=cites:W2...,2018.0,2024.0,2017.0,https://journals.sagepub.com/doi/pdf/10.1177/1...,1.0
3670,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,6.0,False,https://api.openalex.org/works?filter=cites:W2...,2021.0,2024.0,2019.0,NaN,2.0
3671,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,30.0,False,https://api.openalex.org/works?filter=cites:W2...,2013.0,2023.0,2013.0,NaN,0.0
3672,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,1.0,False,https://api.openalex.org/works?filter=cites:W3...,2024.0,2024.0,2020.0,NaN,4.0


In [77]:
df_1 = pd.read_csv('all_items.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = df_doi_1.drop(columns=['DOI'])
df_1 = pd.merge(df_1, df_doi_1, on='Zotero link', how='left')
df_1.to_csv('all_items.csv')

In [78]:
df_2 = pd.read_csv('all_items_duplicated.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = df_doi_2.drop(columns=['DOI'])
df_2 = pd.merge(df_2, df_doi_2, on='Zotero link', how='left')
df_2.to_csv('all_items_duplicated.csv')

In [79]:
df_2 = pd.read_csv('all_items_duplicated.csv')
df_2

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,...,Collection_Link,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,0,Intelligence Officers Have an Ethical Responsi...,Journal article,https://doi.org/10.1080/08850607.2024.2381999,Intelligence officers have an ethical responsi...,https://www.zotero.org/groups/intelligence_bib...,2024-08-29,Cortney Weinbaum,NaN,Intl Journal of Intelligence and Counterintell...,...,https://www.zotero.org/groups/intelligence_bib...,https://openalex.org/W4401989597,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
1,1,12. SOS Hong Kong: Coproducing Espionage Films...,Book chapter,https://www.degruyter.com/document/doi/10.1515...,With the success of D r. No and Goldfinger in ...,https://www.zotero.org/groups/intelligence_bib...,2024-07-10,Sangjoon Lee,Amsterdam University Press,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Counterintelligence Analysis: Catching Spies a...,Book chapter,http://press.georgetown.edu/Book/The-CIA-Intel...,NaN,https://www.zotero.org/groups/intelligence_bib...,2024-09-01,"James B. Bruce, Blake W. Mobley",Georgetown University Press,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Counterintelligence Analysis: Catching Spies a...,Book chapter,http://press.georgetown.edu/Book/The-CIA-Intel...,NaN,https://www.zotero.org/groups/intelligence_bib...,2024-09-01,"James B. Bruce, Blake W. Mobley",Georgetown University Press,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,The CIA Intelligence Analyst: Views from the I...,Book,http://press.georgetown.edu/Book/The-CIA-Intel...,A unique insiders' account of what CIA intelli...,https://www.zotero.org/groups/intelligence_bib...,2024-09-01,"Roger Z. George, Robert Levine",Georgetown University Press,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10145,10145,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10146,10146,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10147,10147,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10148,10148,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
ttt = df_2[df_2['DOI']=='10.1080/08850600600829882']
ttt

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,...,Collection_Link,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
8089,8089,Modeling an Intelligence Analysis Profession o...,Journal article,https://doi.org/10.1080/08850600600829882,NaN,https://www.zotero.org/groups/intelligence_bib...,2006-12-01,"Stephen Marrin, Jonathan D. Clemente",NaN,Intl Journal of Intelligence and Counterintell...,...,https://www.zotero.org/groups/intelligence_bib...,https://openalex.org/W2074061346,17.0,True,https://api.openalex.org/works?filter=cites:W2...,2012.0,2023.0,2006.0,http://www.stuart-hall.com/ftp/IJIC--Vol19No4-...,6.0


In [81]:
# df_zotero_id = df_2[['Zotero link']].copy()
# df_zotero_id['Zotero link'] = df_zotero_id['Zotero link'].str.replace(
#     'https://www.zotero.org/groups/intelligence_bibliography/items/', ''
# )
# df_zotero_id = df_zotero_id.drop_duplicates(subset='Zotero link').reset_index(drop=True)
# df_zotero_id = df_zotero_id.rename(columns={'Zotero link':'zotero_item_key'})
# # df_zotero_id = df_zotero_id.head(100)
# df_zotero_id

In [82]:
df_zotero_id = pd.read_csv('zotero_citation_format.csv')
df_all = pd.read_csv('all_items.csv')
df_all = df_all[['Zotero link']]
df_all['zotero_item_key'] = df_all['Zotero link'].str.replace('https://www.zotero.org/groups/intelligence_bibliography/items/', '')
df_all = df_all.drop_duplicates()
df_not_zotero_id = df_all[~df_all['zotero_item_key'].isin(df_zotero_id['zotero_item_key'])]
df_not_zotero_id = df_not_zotero_id[['zotero_item_key']].reset_index(drop=True)
df_not_zotero_id

C:\Users\yaozk\AppData\Local\Temp\ipykernel_31328\304623864.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.



,zotero_item_key
0,2RBTC347
1,WDKULHDJ
2,7GFMVUQV
3,BZRWDMCP
4,ZDYTUDNP
5,2MH373GH
6,GYK3YWH4
7,GPITSQGD
8,BQ5UN8LI
9,9G9XCZGY


In [83]:
user_id = '2514686'

# Base URL for Zotero API
base_url = 'https://api.zotero.org'

# Initialize an empty string to accumulate bibliographies
all_bibliographies = ""

# List to store bibliographies
bibliographies = []

# Iterate through each item key in the DataFrame
for item_key in df_not_zotero_id['zotero_item_key']:
    # Endpoint to get item bibliography
    endpoint = f'/groups/{user_id}/items/{item_key}'

    # Parameters for the request
    params = {
        'format': 'bib',
        'linkwrap': 1
    }

    # Make GET request to Zotero API
    response = requests.get(base_url + endpoint, params=params)

    # Check if request was successful
    if response.status_code == 200:
        bibliography = response.text.strip()  # Strip any leading/trailing whitespace
        bibliographies.append(bibliography)
        all_bibliographies += f'<p>{bibliography}</p><br><br>'  # Append bibliography with two newlines for separation
    else:
        error_message = f'Error fetching bibliography for item {item_key}: Status Code {response.status_code}'
        bibliographies.append(error_message)
        all_bibliographies += f'<p>{error_message}</p><br><br>'

# Add bibliographies to the original DataFrame
df_not_zotero_id['bibliography'] = bibliographies
df_not_zotero_id

,zotero_item_key,bibliography
0,2RBTC347,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
1,WDKULHDJ,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
2,7GFMVUQV,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
3,BZRWDMCP,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
4,ZDYTUDNP,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
5,2MH373GH,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
6,GYK3YWH4,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
7,GPITSQGD,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
8,BQ5UN8LI,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
9,9G9XCZGY,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."


In [84]:
# print(df_not_zotero_id['bibliography'].iloc[0])

<?xml version="1.0"?>
<div class="csl-bib-body" style="line-height: 1.35; padding-left: 1em; text-indent:-1em;">
  <div class="csl-entry">Weinbaum, Cortney. &#x201C;Intelligence Officers Have an Ethical Responsibility to Use Tradecraft.&#x201D; <i>International Journal of Intelligence and CounterIntelligence</i> 0, no. 0 (August 29, 2024): 1&#x2013;22. <a href="https://doi.org/10.1080/08850607.2024.2381999">https://doi.org/10.1080/08850607.2024.2381999</a>.</div>
</div>


In [85]:
# bibliographies

In [86]:
df_zotero_id = pd.read_csv('zotero_citation_format.csv', index_col=False)
df_zotero_id = df_zotero_id.drop(columns={'Unnamed: 0'})
df_zotero_id = pd.concat([df_zotero_id, df_not_zotero_id]).reset_index(drop=True)
df_zotero_id

,zotero_item_key,bibliography
0,MM95S3LQ,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
1,AN5YI8EM,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
2,UYS2PIFQ,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
3,55Z2ZJDM,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
4,GMC4REMY,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
...,...,...
7260,XRUKR8Z7,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
7261,66K9F7VY,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
7262,TBNI42DW,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
7263,HPUHLU6K,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."


In [87]:
itemL5K2YZ2M = df_zotero_id[df_zotero_id['zotero_item_key']=='JQ6BYCU9']
itemL5K2YZ2M

print(itemL5K2YZ2M['bibliography'].iloc[0])

<?xml version="1.0"?>
<div class="csl-bib-body" style="line-height: 1.35; padding-left: 1em; text-indent:-1em;">
  <div class="csl-entry">Ischebeck-Baum, Frederic. &#x201C;Spies at the Heart of the Cold War: The British Commanders&#x2019;-In-Chief Mission to the Soviet Forces in Germany 1946 &#x2013; 1990 (&#x2018;BRIXMIS&#x2019;).&#x201D; PhD Thesis, King&#x2019;s College London, 2020. <a href="https://kclpure.kcl.ac.uk/portal/en/studentTheses/spies-at-the-heart-of-the-cold-war">https://kclpure.kcl.ac.uk/portal/en/studentTheses/spies-at-the-heart-of-the-cold-war</a>.</div>
</div>


In [88]:
df_zotero_id.to_csv('zotero_citation_format.csv')